# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

My understanding is that the ask is to create an intervention system that can identify when a student is having issues that might lead them to fail to graduate. Given this context, I'd approach this challenge as a classification challenge, since we would need to understand multiple features (input values) to flag if an intervention is needed or not, meaning that we need to map it to a discrete output. In this case, we don't have a continuous possible outcome, but rather a discrete set of output values, which would be something like "needs intervention" or "doesn't need intervention".

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [7]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [8]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)

# TODO: Calculate passing students
n_passed = sum(student_data.passed == 'yes')

# TODO: Calculate failing students
n_failed = sum(student_data.passed == 'no')

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / float(n_students)

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 0.67%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [9]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  higher internet  romantic  famrel  freetime goout Dalc Walc health absences  
0    yes       no        no       4         3     4    1    1     

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [10]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
#X_train = None
#X_test = None
#y_train = None
#y_test = None
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=num_test, train_size=num_train, random_state=58)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Anwser**

I’ve spent time researching all the different algorithms, looking at common applications, their strengths and weaknesses, taking into account the current project context and data. I’ve explored scikit-learn documentation, Wikipedia and several blogs, along with the Udacity course content, resources and my own notes. Based on that research, I’ve chosen to move forward with the following three algorithms:

**Ensemble Boosting - AdaBoost
Support Vector Machines (SVM)
Logistic Regression**

**Ensemble Boosting - AdaBoost:** The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning algorithm in order to improve generalizability / robustness over a single estimator. The core principle of AdaBoost is to fit a sequence of weak learners (i.e., models that are only slightly better than random guessing, such as small decision trees) on repeatedly modified versions of the data. The predictions from all of them are then combined through a weighted majority vote (or sum) to produce the final prediction. Ensemble techniques can be used in combination with other algorithms like Decision Trees, Random Forest, AdaBoost or Gradient Boosting. 

-**Real World Examples:** They are sometimes used in Loan Prediction applications, or predictions for Age or Income. Ensemble Learning techniques are often used in Fraud Detection, Malware Detection, Medicine (detection of diseases based on MRI datasets), Face Recognition and Speech Recognition among others. Ensemble methods have been very successful in setting record performance on challenging datasets and are among the top winners of Kaggle data science competitions.

-**Strengths:** Computationally efficient. No difficult parameters to set. Versatile ­ a wide range of base learners can be used with AdaBoost. It’s not prone to overfitting and can help to minimize noise, bias and variance. Overall, it provides More accurate prediction results. Ensemble methods are good when we are not sure which features to use, since it uses a sample of different features / questions to find the best performing ones. 

-**Weaknesses:** Seems susceptible to uniform noise. Weak learners should not be too complex ­to avoid overfitting. There needs to be enough data so that the weak learning requirement is satisfied. Adaboost can be slow to train.

-**Selection Justification:** Seems that ensemble learning models promote diversity of models, and offer a generally more accurate output. They are often used in classification problems, especially when there might be no concrete understanding over which features might be better. Real world applications seem to be close to the current problem we are trying to solve. I’m also curious to try out their accuracy, against commonly used algorithms like Random Forest and Base Decision Trees.

**Support Vector Machines (SVM):** Support Vector Machines are an algorithm that helps us to find the best line that separates our data by maximising the margin. They are similar to instance based ML, in particular the nearest neighbour algorithm, but we do some work to find just the neighbours that matter. When we don’t have linearly separable data, we can look into the concept of similarity (how similar is a point to another) and use what’s called the Kernel Trick, which through some clever math, allows us to move points to different dimensions, without modifying the data, creating a way for us to separate the data.

-**Real World Examples:** SVMs have been used across applications like handwriting recognition (recognizing handwritten characters used widely), generalized predictive control (control chaotic dynamics with useful parameters) and text and hypertext categorization (text and hypertext categorization for both inductive and transductive models)

-**Strengths:** Effective in high dimensional spaces. Still effective in cases where the number of dimensions is greater than the number of samples. Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient. Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

-**Weaknesses:** Tendency to overfit. If the number of features is much greater than the number of samples, avoiding over-fitting in choosing Kernel functions and regularization term is crucial. SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation. Their compute and storage requirements increase rapidly with the number of training vectors. They might not be able to properly model feature correlations. 

-**Selection Justification:** SVMs perform well at classification problems with high dimension spaces. Since we don’t know which features are the important ones, and what are their inter-relationships, I’m curious to see how well the algorithm performs. Also, although we have a lot of features, the amount of data is more than the features, which should make it work well enough. Although I was thinking of going for Gaussian Naive Bayes, since SVMs might perform better given our data, I wanted to give them a try and compare their performance. 

**Logistic Regression:** Logistic regression, despite its name, is a linear model for classification rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function. In statistics, the logistic model (or logit model) is used to model the probability of a certain class or event existing such as pass/fail, win/lose, alive/dead or healthy/sick. This can be extended to model several classes of events such as determining whether an image contains a cat, dog, lion, etc.

-**Real World Examples:** Logistic regression is used in various fields, including machine learning, most medical fields, and social sciences. For example, the Trauma and Injury Severity Score (TRISS), which is widely used to predict mortality in injured patients, was originally developed by Boyd et al. using logistic regression. Logistic regression may be used to predict the risk of developing a given disease (e.g. diabetes; coronary heart disease), based on observed characteristics of the patient (age, sex, body mass index, results of various blood tests, etc.). Another example might be to predict whether a Nepalese voter will vote Nepali Congress or Communist Party of Nepal or Any Other Party, based on age, income, sex, race, state of residence, votes in previous elections, etc. It is also used in marketing applications such as prediction of a customer's propensity to purchase a product or halt a subscription, etc. Other examples are predicting if an email is spam or not spam, whether a tumor is malignant or benign, and whether a mushroom is poisonous or edible.

-**Strengths:** powerful machine learning algorithm that utilizes a sigmoid function and works best on binary classification problems. It’s simple. It’s easy to understand, interpretable, and can give pretty good results.It is the go-to method for binary classification problems (problems with two class values).

-**Weaknesses:** Logistic regression (despite its name) is not fit for regression tasks. Sensible to outliers and noisy data. It does assume a linear relationship between the input variables with the output. Like linear regression, the model can overfit if you have multiple highly-correlated inputs. 

-**Selection Justification:** Logistic regression is most commonly used when the data in question has binary output, so when it belongs to one class or another, or is either a 0 or 1, which makes it a good fit for this particular case. It can deal with feature correlation, unlike Naive Bayes and SVMs. 

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [12]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A - AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# from sklearn import model_B - SVC
from sklearn.svm import SVC

# from sklearn import model_C - Logistic Regression
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = AdaBoostClassifier(n_estimators=100, random_state=42)
clf_B = SVC(random_state=42, gamma='auto')
clf_C = LogisticRegression(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train.iloc[:100, :]
y_train_100 = y_train.iloc[:100]

X_train_200 = X_train.iloc[:200, :]
y_train_200 = y_train.iloc[:200]

X_train_300 = X_train.iloc[:300, :]
y_train_300 = y_train.iloc[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

X_train_list = [X_train_100, X_train_200, X_train_300]
y_train_list = [y_train_100, y_train_200, y_train_300]

# loop through the three classifiers
for clf in [clf_A, clf_B, clf_C]:
    
    # show the estimator type
    print("\n{}: \n".format(clf.__class__.__name__))

    # loop through training lists, and execute the predict function
    for X_train_set, y_train_set in zip(X_train_list, y_train_list):
        train_predict(clf, X_train_set, y_train_set, X_test, y_test)


AdaBoostClassifier: 

Training a AdaBoostClassifier using a training set size of 100. . .
Trained model in 0.2812 seconds
Made predictions in 0.0103 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0115 seconds.
F1 score for test set: 0.6885.
Training a AdaBoostClassifier using a training set size of 200. . .
Trained model in 0.2997 seconds
Made predictions in 0.0128 seconds.
F1 score for training set: 0.9603.
Made predictions in 0.0101 seconds.
F1 score for test set: 0.7344.
Training a AdaBoostClassifier using a training set size of 300. . .
Trained model in 0.2559 seconds
Made predictions in 0.0177 seconds.
F1 score for training set: 0.8756.
Made predictions in 0.0137 seconds.
F1 score for test set: 0.7227.

SVC: 

Training a SVC using a training set size of 100. . .
Trained model in 0.0026 seconds
Made predictions in 0.0026 seconds.
F1 score for training set: 0.8987.
Made predictions in 0.0018 seconds.
F1 score for test set: 0.6765.
Training a SVC using a training 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - AdaBoostClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.2812                  | 0.0115                 | 1.0000           | 0.6885          |
| 200               | 0.2997                  | 0.0101                 | 0.9603           | 0.7344          |
| 300               | 0.2559                  | 0.0137                 | 0.8756           | 0.7227          |

** Classifer 2 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0024                  | 0.0018                 | 0.8987           | 0.6765          |
| 200               | 0.0043                  | 0.0042                 | 0.8817           | 0.7133          |
| 300               | 0.0043                  | 0.0030                 | 0.8689           | 0.7391          |

** Classifer 3 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0024                  | 0.0015                 | 0.9041           | 0.7130          |
| 200               | 0.0047                  | 0.0014                 | 0.8812           | 0.7031          |
| 300               | 0.0043                  | 0.0017                 | 0.8640           | 0.7187          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

To my surprise, SVC (Support Vector Machine) performed better than the other two algorithms. With a relatively small set of data, the F1 score for the test set with a training set of 300 instances returned the best score out of the three algorithms. Performance improved as more data was added, the training time and the prediction time are really fast compared to AdaBoost, and pretty much as fast as LogisticRegression. Seems that the ability to work with high dimension spaces with no aparent feature correlation works well in this example.  

I was tempted to choose AdaBoost as the best algorithm, since it seems that it worked best at 200 samples, and it's supposed to be a really good algorithm (Ensemble Learning Boosting), and there could be the potential indication that SVM is just overfitting as more data is added to the training set. However, the F1 scoring on the training of SVM seems to fall as more data was added, giving me a hint that overfitting is not a problem. 


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Support Vector Machines is an algorithm that helps us find the best line that separates the data and classifies it correctly, by maximising the margin (distance to the closest points on both sides of the line). 

SVMs use the Kernel trick, which allows points to be moved into different dimensions without altering the data, enabling us to find ways to separate the data using hyperplanes across different dimensions. 

In simple terms, the Kernel trick allows us take a non-linearly separable problem, and add more dimensions to it, using what is called kernels. This allows us to move to a space where the data is linearly separable. We then find the solution that separates the data correctly, and that same solution allows us to take that solution back to the non-separable data space. 

This enables the algorithm to work well on high dimension spaces, or when we have many features without an apparent 
correlation. SVMs puts the correct classification of the data first, and then maximises the data, meaning tries to maximise the margin.

C and Gamma are important parameters to tune the algorithm. Another consideration is that SVMs tend to overfit, so these parameters help to control that case.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [42]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedShuffleSplit

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}

parameters_ada = {'n_estimators' : [30, 50, 70, 80],'learning_rate': [0.001, 0.01, .80, 1.0]}

# TODO: Initialize the classifier
clf =  SVC()
clf_ada = AdaBoostClassifier(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# Create Stratified ShuffleSplit cross-validator
ssscv = StratifiedShuffleSplit(n_splits=5, test_size=0.24, random_state=42)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer, cv=ssscv)
grid_obj_ada = GridSearchCV(clf_ada, parameters_ada, scoring=f1_scorer, cv=ssscv)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
grid_obj_ada = grid_obj_ada.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
clf_ada = grid_obj_ada.best_estimator_

# Print the parameters
print clf.get_params(), '\n'

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

print '\n'
print clf_ada.get_params(), '\n'

print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf_ada, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf_ada, X_test, y_test))

{'kernel': 'rbf', 'C': 100, 'verbose': False, 'probability': False, 'degree': 3, 'shrinking': True, 'max_iter': -1, 'decision_function_shape': 'ovr', 'random_state': None, 'tol': 0.001, 'cache_size': 200, 'coef0': 0.0, 'gamma': 0.0001, 'class_weight': None} 

Made predictions in 0.0046 seconds.
Tuned model has a training F1 score of 0.8507.
Made predictions in 0.0024 seconds.
Tuned model has a testing F1 score of 0.7153.


{'n_estimators': 30, 'base_estimator': None, 'random_state': 42, 'learning_rate': 0.01, 'algorithm': 'SAMME.R'} 

Made predictions in 0.0045 seconds.
Tuned model has a training F1 score of 0.8484.
Made predictions in 0.0035 seconds.
Tuned model has a testing F1 score of 0.7338.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

For SVC, the final F1 scores are:
F1 Score Training: 0.8507
F1 Score Testing: 0.7153

Both scores are lower than training and testing the untunned model with 300 samples, but the tuned F1 score is slightly better than the untunned model with 200 samples. I suspect that the tuned model is selecting the best parameters that prevent overfitting, while genrating the best possible model and combination of parameters.

After doing this, I was curious to test the AdaBoost algorithm with tuning, and see how it performed. As it turns out, The tuned AdaBoost has lower training score, but slightly better F1 testing score, and it performs better than the tuned SVC (although it has a lower score than the SVC untunned model trained with 300 samples). Also the training and testing times of the tuned AdaBoost are just slightly higher than the tuned SVC, and way better than the untunned AdaBoost. This might point out that at the end, the tuned AdaBoost is preventing overfitting while performing better than SVC for this particular case.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.